In [1]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

iris_data=load_iris()

In [2]:
# Features names and target values
print("Feature names:", iris_data.feature_names)
print("First 5 rows of features:\n", iris_data.data[:5])
print("\nTarget names (species):", iris_data.target_names)
print("First 5 target values:", iris_data.target[:5])

Feature names: ['sepal length (cm)', 'sepal width (cm)', 'petal length (cm)', 'petal width (cm)']
First 5 rows of features:
 [[5.1 3.5 1.4 0.2]
 [4.9 3.  1.4 0.2]
 [4.7 3.2 1.3 0.2]
 [4.6 3.1 1.5 0.2]
 [5.  3.6 1.4 0.2]]

Target names (species): ['setosa' 'versicolor' 'virginica']
First 5 target values: [0 0 0 0 0]


In [3]:
# We split the data into training and testing sets

X_train, X_test, y_train, y_test = train_test_split(iris_data.data, iris_data.target, test_size=0.8, random_state=42)

In [4]:
# We can test a prediction model
# We are going to use SVM:

from sklearn import svm
from sklearn.metrics import accuracy_score

model=svm.SVC(probability=True)
model.fit(X_train,y_train)
y_pred=model.predict_proba(X_test)
print(f"Probability of each class:\n {y_pred}")
y_pred=model.predict(X_test)
accuracy=accuracy_score(y_pred,y_test)
print(f"Accuracy: {accuracy}")

Probability of each class:
 [[0.03346461 0.70229104 0.26424434]
 [0.782712   0.15750736 0.05978064]
 [0.05128086 0.0287268  0.91999234]
 [0.03517948 0.70280313 0.26201739]
 [0.03593386 0.56961275 0.3944534 ]
 [0.81681291 0.13059073 0.05259636]
 [0.08566389 0.8434342  0.07090191]
 [0.03445142 0.17755625 0.78799233]
 [0.03239241 0.72756248 0.24004511]
 [0.05746133 0.86201226 0.08052641]
 [0.03249582 0.24384806 0.72365612]
 [0.85011205 0.10254605 0.0473419 ]
 [0.85126868 0.10134301 0.04738831]
 [0.8394361  0.11189603 0.04866787]
 [0.83481215 0.11241429 0.05277356]
 [0.03734263 0.54829622 0.41436115]
 [0.02677959 0.06742568 0.90579473]
 [0.05906929 0.86696338 0.07396733]
 [0.03571455 0.75494999 0.20933546]
 [0.02611245 0.09858399 0.87530356]
 [0.82475934 0.12260491 0.05263575]
 [0.03107517 0.41784278 0.55108206]
 [0.81255618 0.13307664 0.05436718]
 [0.02623369 0.10889809 0.86486822]
 [0.05693134 0.04423996 0.8988287 ]
 [0.03131278 0.15782984 0.81085738]
 [0.02771307 0.11315969 0.85912724]


The model seems to be extremely efficient to predict the type of petal since the probabilities are really distributed

We are going to save our model in a joblib file

In [5]:
from joblib import dump,load

dump(model,'iris_model.joblib')

['iris_model.joblib']

In [6]:
from flask import Flask, request, jsonify
from joblib import load
import numpy as np

app = Flask(__name__)

# Load the model
model = load('iris_model.joblib')

# Make sure to load the iris dataset for target_names (assuming it's not already loaded)
from sklearn.datasets import load_iris
iris_data = load_iris()

@app.route('/predict', methods=['GET'])
def predict():
    try:
        # Extract features from query parameters
        features = [float(request.args.get(feat)) for feat in ['sepal_length', 'sepal_width', 'petal_length', 'petal_width']]
        
        # Make prediction - Note the change here to wrap features in another list
        prediction = model.predict([features])[0]
        probas = model.predict_proba([features])[0]  # Wrap features in another list
        species = iris_data.target_names[prediction]

        probabilities_dict = {iris_data.target_names[i]: probas[i] for i in range(len(probas))}

        # Standardized API response format
        response = {
            "prediction": species,
            "probabilities": probabilities_dict
        }

    except Exception as e:
        response = {
            "message": str(e)
        }

    return jsonify(response)

if __name__ == '__main__':
    app.run(debug=False)


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [12/Feb/2024 15:38:09] "GET /predict?sepal_length=5.1&sepal_width=3.5&petal_length=1.4&petal_width=0.2 HTTP/1.1" 200 -
127.0.0.1 - - [12/Feb/2024 15:39:46] "GET /predict?sepal_length=5.1&sepal_width=3.5&petal_length=1.4&petal_width=0.2 HTTP/1.1" 200 -
